In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.6.1


In [2]:
!python -V

Python 3.10.12


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/dan/environments/.env_dataeng/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dan/environments/.env_dataeng/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
year = 2023
month = 3
    
# Read data
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year:04d}-{month:02d}.parquet')

In [ ]:
# df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_????-??.parquet')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [9]:
import numpy as np
 # Create ride_id
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
# Create result dataframe
df_result = pd.DataFrame({
    'ride_id': df['ride_id'],
    'predicted_duration': y_pred
})
    
# Save results
output_file = f'predictions_{year:04d}_{month:02d}.parquet'
df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )
    
print(f"Standard deviation: {np.std(y_pred)}")


Standard deviation: 6.247488852238703


In [10]:
std_dev = y_pred.std()
print(std_dev)

6.247488852238703


In [ ]:
import os
# Your code to save the file
output_file = f'predictions_{year:04d}_{month:02d}.parquet'
df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )


file_size_bytes = os.path.getsize(output_file)
print(f"The file size is: {file_size_bytes} bytes")

# Convert bytes to megabytes (MB) for the homework question
file_size_mb = file_size_bytes / (1024 * 1024)
print(f"The file size is: {file_size_mb:.2f} MB")                                                                                       

The file size is: 68640926 bytes
The file size is: 65.46 MB


In [ ]:
from pathlib import Path

file_path = Path(output_file)
file_size_bytes = file_path.stat().st_size
print(f"The file size is: {file_size_bytes} bytes")

file_size_mb = file_size_bytes / (1024 * 1024)
print(f"The file size is: {file_size_mb:.2f} MB")

The file size is: 68640926 bytes
The file size is: 65.46 MB


In [1]:
!cat Pipfile.lock | grep -A 3 scikit-learn


        "scikit-learn": {
            "hashes": [
                "sha256:057b991ac64b3e75c9c04b5f9395eaf19a6179244c089afdebaad98264bff37c",
                "sha256:118a8d229a41158c9f90093e46b3737120a165181a1b58c03461447aa4657415",
